In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.cluster import KMeans
df  = pd.read_csv('/kaggle/input/world-happiness-report-2021/world-happiness-report-2021.csv',index_col=0)
cols = list(df.columns)[5:11]
df = df[cols]
df.tail()

# df.drop(columns=['Regional indicator','Ladder score','Standard error of ladder score','upperwhisker'])

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(df)


In [ ]:
df_ = df[df.index == 'Uzbekistan']
df_

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
merging = linkage(X, method='ward')
plt.figure(figsize=(20,20))
# Строим дендрограмму
dendrogram(merging, labels=df.index, leaf_font_size=10)
plt.show()

In [ ]:
clusters = fcluster(merging, 15, criterion='distance')
print(clusters)

In [ ]:
# Кластеризация методом k-средних на 3 кластера
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
labels = kmeans.predict(X)

# Вывод результатов в виде словаря
print(dict(zip(df.index, labels)))

In [ ]:
# Подберём наилучшее количество кластеров ("правило локтя")

# crit = []
# for k in range(2, 7):
#     kmeans = KMeans(n_clusters=k, random_state=15)
#     kmeans.fit(X)
#     crit.append(kmeans.inertia_)
    
# plt.plot(range(2,7), crit)
# plt.show()

# Добавляем фичу, которая описывает в какой класс входит фича

In [ ]:
df['cluster'] = kmeans.labels_
print(set(kmeans.labels_))
df.tail()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
pca.fit(X)
Z = pca.transform(X)

In [ ]:
print(Z)

In [ ]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)#здесь видим, какой процент информации находится в какой компоненте

In [ ]:
plt.plot(Z[labels == 0,0], Z[labels == 0,1], 'bo', label='Clauster 1') 
plt.plot(Z[labels == 1,0], Z[labels == 1,1], 'go', label='Clauster 2') 
plt.plot(Z[labels == 2,0], Z[labels == 2,1], 'ro', label='Clauster 3') 
plt.plot(Z[labels == 3,0], Z[labels == 3,1], 'yo', label='Clauster 4') 
plt.legend(loc=4)

In [ ]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

loading_matrix = pd.DataFrame(loadings, columns=['PC1', 'PC2'], index=df.columns[:-1])
loading_matrix.sort_values(by=['PC1'],inplace=True)
print(loading_matrix)
loading_matrix.sort_values(by=['PC2'],inplace=True)
print(loading_matrix)

In [ ]:
#Смотреть график выше
#исходим из данных выше, ось X - развитость экономики, ось Y - щедрость/доброта людей
print(df.index[np.argmin(Z[:,1])])#самые злые люди
print(df.index[np.argmax(Z[:,0])]) #самая экономически неразвитая страна

In [ ]:
print(df.index[np.argmin(Z[:,0])]) # самая экономически развитая страна 